In [1]:
# Copyright (c) 2024 Microsoft Corporation.
# Licensed under the MIT License.

In [2]:
import os

import pandas as pd
import tiktoken

from graphrag.config.enums import ModelType
from graphrag.config.models.language_model_config import LanguageModelConfig
from graphrag.language_model.manager import ModelManager
from graphrag.query.indexer_adapters import (
    read_indexer_communities,
    read_indexer_entities,
    read_indexer_reports,
)
from graphrag.query.structured_search.global_search.community_context import (
    GlobalCommunityContext,
)
from graphrag.query.structured_search.global_search.search import GlobalSearch

## Local Search Example

Local search method generates answers by combining relevant data from the AI-extracted knowledge-graph with text chunks of the raw documents. This method is suitable for questions that require an understanding of specific entities mentioned in the documents (e.g. What are the healing properties of chamomile?).

### Load text units and graph data tables as context for local search

- In this test we first load indexing outputs from parquet files to dataframes, then convert these dataframes into collections of data objects aligning with the knowledge model.

### Load tables to dataframes

In [3]:
#MODEL = "llama-3.3-70b-versatile"
MODEL = "gpt-4o-mini"
from dotenv import load_dotenv
load_dotenv('.env')

groq_api_key = os.environ["GROQQ_API_KEY"]
qwen_api_key = os.environ["QWEN_API_KEY"]
ds_api_key = os.environ["DEEPSEEK_API_KEY"]
api_key = os.environ["GRAPHRAG_API_KEY"]
llm_model = os.environ["GRAPHRAG_LLM_MODEL"]

config = LanguageModelConfig(
    api_key=api_key,
    #api_base="https://api.deepseek.com/v1",
    #api_base="https://api.groq.com/openai/v1",
    api_base="https://api.openai.com/v1",
    type=ModelType.OpenAIChat,
    model=MODEL,
    max_retries=2,
    encoding_model="cl100k_base"
)
model = ModelManager().get_or_create_chat_model(
    name="global_search",
    model_type=ModelType.OpenAIChat,
    config=config,
)

token_encoder = tiktoken.get_encoding("cl100k_base")

In [4]:
# parquet files generated from indexing pipeline
INPUT_DIR = f"./output/{MODEL}"
COMMUNITY_TABLE = "communities"
COMMUNITY_REPORT_TABLE = "community_reports"
ENTITY_TABLE = "entities"

# community level in the Leiden community hierarchy from which we will load the community reports
# higher value means we use reports from more fine-grained communities (at the cost of higher computation cost)
COMMUNITY_LEVEL = 2

#### Read entities

In [5]:
community_df = pd.read_parquet(f"{INPUT_DIR}/{COMMUNITY_TABLE}.parquet")
entity_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_TABLE}.parquet")
report_df = pd.read_parquet(f"{INPUT_DIR}/{COMMUNITY_REPORT_TABLE}.parquet")

communities = read_indexer_communities(community_df, report_df)
reports = read_indexer_reports(report_df, community_df, COMMUNITY_LEVEL)
entities = read_indexer_entities(entity_df, community_df, COMMUNITY_LEVEL)

print(f"Total report count: {len(report_df)}")
print(
    f"Report count after filtering by community level {COMMUNITY_LEVEL}: {len(reports)}"
)

report_df.head()

Total report count: 221
Report count after filtering by community level 2: 221


,id,human_readable_id,community,level,parent,children,title,summary,full_content,rank,rating_explanation,findings,full_content_json,period,size
0,e199ef0aa7934ce19073a7572620eac2,182,182,2,20,[],Mealworms and Plastic Biodegradation Research ...,This community focuses on the biodegradation o...,# Mealworms and Plastic Biodegradation Researc...,9.5,The community's research on mealworms' biodegr...,"[{'explanation': 'Frass, the excrement of Tene...","{\n ""title"": ""Mealworms and Plastic Biodegr...",2025-03-20,28
1,2cefccd402ca4bc08e650091ab3a4988,183,183,2,20,[],Plastic Biodegradation Community: Mealworms an...,This community focuses on the research surroun...,# Plastic Biodegradation Community: Mealworms ...,9.5,The impact severity rating is high due to the ...,[{'explanation': 'Gas Chromatography-Mass Spec...,"{\n ""title"": ""Plastic Biodegradation Commun...",2025-03-20,2
2,fbf63ab9284b4acea7abb82ecbaa1a36,184,184,2,20,[],Mealworm Biodegradation Research Community,The community focuses on the interactions betw...,# Mealworm Biodegradation Research Community\n...,8.5,The rating reflects the significant relevance ...,[{'explanation': 'Ion chromatography is a cruc...,"{\n ""title"": ""Mealworm Biodegradation Resea...",2025-03-20,2
3,36f60c36a0114702be14584f29c6645f,185,185,2,22,[],Mealworms and PVC Biodegradation Community,The community focuses on the biodegradation of...,# Mealworms and PVC Biodegradation Community\n...,9.0,The rating reflects the significant potential ...,[{'explanation': 'Research indicates that Tene...,"{\n ""title"": ""Mealworms and PVC Biodegradat...",2025-03-20,5
4,a8af729af2714a31815be93c83aeb403,186,186,2,22,[],Mealworms and Plastic Biodegradation Research ...,This community centers around the research on ...,# Mealworms and Plastic Biodegradation Researc...,9.0,The impact severity rating is high due to the ...,[{'explanation': 'Depolymerization is essentia...,"{\n ""title"": ""Mealworms and Plastic Biodegr...",2025-03-20,3


#### Build global context based on community reports

In [6]:
context_builder = GlobalCommunityContext(
    community_reports=reports,
    communities=communities,
    entities=entities,  # default to None if you don't want to use community weights for ranking
    token_encoder=token_encoder,
)

#### Perform global search

In [7]:
context_builder_params = {
    "use_community_summary": False,  # False means using full community reports. True means using community short summaries.
    "shuffle_data": True,
    "include_community_rank": True,
    "min_community_rank": 0,
    "community_rank_name": "rank",
    "include_community_weight": True,
    "community_weight_name": "occurrence weight",
    "normalize_community_weight": True,
    "max_tokens": 12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
    "context_name": "Reports",
}

map_llm_params = {
    "max_tokens": 1000,
    "temperature": 0.6,
    "response_format": {"type": "json_object"},
}

reduce_llm_params = {
    "max_tokens": 2000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 1000-1500)
    "temperature": 0.6,
}

In [8]:
search_engine = GlobalSearch(
    model=model,
    context_builder=context_builder,
    token_encoder=token_encoder,
    max_data_tokens=12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
    map_llm_params=map_llm_params,
    reduce_llm_params=reduce_llm_params,
    allow_general_knowledge=True,  # set this to True will add instruction to encourage the LLM to incorporate general knowledge in the response, which may increase hallucinations, but could be useful in some use cases.
    json_mode=True,  # set this to False if your LLM model does not support JSON mode.
    context_builder_params=context_builder_params,
    concurrent_coroutines=32,
    response_type="multiple paragraphs",  # free form text describing the response type and format, can be anything, e.g. prioritized list, single paragraph, multiple paragraphs, multiple-page report
)

In [9]:
async def get_answer(df, type):
    answers = []
    for query in df['Query']:
        if not pd.isna(query):
            print(query)
            answer = await search_engine.search(query)
            answers.append(answer.response)

    df['answer'] = answers
    directory = f'./answers/{MODEL}/'
    os.makedirs(directory, exist_ok=True)
    df.to_excel(f"{directory}{MODEL} {type} global output.xlsx", index=False)

for type in ["Text", "TF", "Numerical", "Exp Design"]:
    df = pd.read_excel('../Query.xlsx', sheet_name=type, header=0)
    await get_answer(df, type)

What kind of hydrolases were reported in transcriptome analysis to be upregulated during the biodegradation of LDPE by Tenebrio molitor larvae? Please give answers in one short paragraph.
What specific compounds were reported in GC-MS analysis as degradation products of aged PE in the frass of mealworm larvae? Please list all of them in one short paragraph.
What were the main changes of functional groups of PE mulching films treated by coculture of Acinetobacter sp. strain NyZ450 and Bacillus sp. strain NyZ451 as reported? Please list all changes in term of peaks at the FTIR spectra in one short paragraph.
What bacterial genera were reported to be significantly upregulated in PE + bran diet group, compared with the sole bran diet group? Please only consider the mealworm gut microbiome studies for LDPE foam biodegradation and give answers in one short paragraph.
What bacterial genera were reported to be predominant (top 3 average relative abundance in the microbiome studies) in the gut 